In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [3]:
movies = pd.read_csv('ml-1m/movies.dat',sep='::',header= None, engine = 'python',encoding='latin-1')
users=pd.read_csv('ml-1m/users.dat',sep='::',header= None, engine = 'python',encoding='latin-1')
ratings=pd.read_csv('ml-1m/ratings.dat',sep='::',header= None, engine = 'python',encoding='latin-1')

In [5]:
training_set=pd.read_csv('ml-100k/u1.base',delimiter='\t')
training_set = np.array(training_set,dtype='int')
test_set=pd.read_csv('ml-100k/u1.test',delimiter='\t')
test_set=np.array(test_set,dtype='int')

In [28]:
nb_users = int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies =int(max(max(training_set[:,1]),max(test_set[:,1])))

In [22]:
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [30]:
def convert(data):
    new_data=[]
    for id_users in range(1,nb_users+1):
        id_movies = data[:,1][data[:,0]==id_users]
        id_ratings = data[:,2][data[:,0]==id_users]
        rating = np.zeros(nb_movies)
        rating[id_movies-1]=id_ratings
        new_data.append(list(rating))
    return new_data       

In [31]:
training_set=convert(training_set)
test_set=convert(test_set)

In [32]:
training_set=torch.FloatTensor(training_set)
test_set=torch.FloatTensor(test_set)

In [35]:
class SAE(nn.Module):
    def __init__(self,):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20,10)
        self.fc3 = nn.Linear(10,20)
        self.fc4 = nn.Linear(20,nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.activation(self.fc4(x))
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(),lr = 0.01, weight_decay = 0.5)     

In [36]:
nb_epoch = 200

In [42]:
for epoch in range(1,nb_epoch +1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data>0)>0:
            output = sae(input)
            target.require_grad = False
            output[target ==0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data>0) +1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch:'+str(epoch)+' loss:'+str(train_loss/s))

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [1, 1682]], which is output 0 of SigmoidBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).